# Assistant with File-search ability

In [43]:
import dotenv
from openai import OpenAI

In [44]:
dotenv.load_dotenv(".env")

MODEL="gpt-4o-mini"

In [45]:
client = OpenAI()

## Step 1: Create a new Assistant with File Search Enabled

In [46]:
# Create a new assistant 
assistant = client.beta.assistants.create(
  name="Sloboda documents assistant",
  instructions="You are an helpful assistant to help users find information in Sloboda Rules documents.",
  model=MODEL,
  tools=[{"type": "file_search"}],
)

## Step 2: Upload files and add them to a Vector Store

In [47]:
# Create a vector store caled "Financial Statements"
vector_store = client.beta.vector_stores.create(name="Sloboda Rules")
 
# Ready the files for upload to OpenAI
file_paths = [
  "data/raw/Hardware provision policy.pdf",
  "data/raw/Sunflower game.pdf",
  "data/raw/Vacation policy.pdf",
]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=3, failed=0, in_progress=0, total=3)


## Step 3: Update the assistant to use the new Vector Store

In [48]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

## Step 4: Create a thread and message

In [49]:
thread = client.beta.threads.create()

In [50]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is Sunflower game?",
)

## Step 5: Create a run and check the output

In [51]:
# Use the create and poll SDK helper to create a run and poll the status of
# the run until it's in a terminal state.

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, 
    assistant_id=assistant.id,
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

In [52]:
# message
messages[0].content[0].text.value

'The Sunflower Game is an initiative at Sloboda Studio designed to reward employees for their contributions and creativity. In this game, team members can earn "Sunflower Points" (☀) by participating in various activities aimed at improving company processes and encouraging positive interactions. These points can later be exchanged for rewards from a prize pool.\n\n### How It Works:\nEmployees can earn points across different categories, including:\n\n- **Company Development Idea**: Propose new processes or tools (10☀).\n- **Project Enhancement Idea**: Suggest features or solutions (10☀).\n- **Client Appreciation**: Earned for outstanding work (15☀).\n- **Peer Appreciation**: For helping colleagues (10☀).\n- **Mentorship**: For guiding other team members (10☀).\n- **Candidate Referral**: For recommending new hires (20☀).\n- **Marketing Support**: Assisting the marketing team (20☀).\n- **Public Speaking**: Representing the company at events (50☀).\n- **Writing Articles**: Contributing c

In [53]:
# tokens used
run.usage

Usage(completion_tokens=317, prompt_tokens=3147, total_tokens=3464)

## Create another message in the same thread

In [58]:
client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can I get a free notebook from Sloboda?",
    # content="How are you?",
    # content="How can you hep me with sloboda rules?",
)
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
messages[0].content[0].text.value

"Yes, you can obtain a notebook or other hardware from Sloboda, but it depends on your eligibility and whether corporate hardware is available. Here are the options available to you:\n\n1. **Using Your Personal Device**: If you opt to use your own device, you may receive a compensation amortization monthly, with specific amounts allocated based on your role.\n\n2. **Requesting Corporate Hardware**: If you cannot use your device, you can request corporate hardware. The process involves:\n   - Contacting HR to check for availability.\n   - Signing an agreement regarding the use of the hardware.\n   - Providing your delivery address.\n\n3. **Buying New Hardware**: If corporate hardware isn't available and you wish to buy a new device, the company can assist you in purchasing it. However, you must have worked in the company for at least one month, and the costs should not exceed 50% of your monthly compensation【4:0†source】."

In [59]:
run.usage

Usage(completion_tokens=196, prompt_tokens=1289, total_tokens=1485)